In [ ]:
import cv2
from sklearn.utils import shuffle
from Utils import *
import matplotlib.pyplot as plt

Read in images and shuffle

In [ ]:
img_folder = "./data/images/dset/gt"
all_images = read_in_images_simple(folder=img_folder)
all_images = shuffle(all_images)

ds_len = len(all_images)
train_end_index = int(ds_len * 0.8)
val_end_index = int(ds_len * 0.9)

train_gt = all_images[:train_end_index]
val_gt = all_images[train_end_index:val_end_index]
test_gt = all_images[val_end_index:]



In [ ]:
print(f" Number of training images: {len(train_gt)}")
print(f" Number of validation images: {len(val_gt)}")
print(f" Number of test images: {len(test_gt)}")
